## 1.3. Criptografia Inquebrável

Para criar essa criptografia, o autor usa a ideia de `ont-time pad`, ou cifra de uso único, que é uma forma de criptografar dados combinando eles com dados aleatórios e sem significado, chamados pelo autor de `dummy`. Com essa ideia, os dados originais só poderão ser descriptografados com o produto da criptografia e o `dummy`, remetendo a ideia de criptografias que usam um par de chaves onde obter apenas uma delas é inútil para acessar os dados originais.

O autor coloca como boa prática para a criação dos `dummys` três pontos para que a criptografia se torne de fato robusta:

> 1 - Devem ter o mesmo tamanho dos dados que serão criptografados;
>
> 2 - Devem ser de fato aleatórios (ou [pseudoaletórios](https://pt.wikipedia.org/wiki/Pseudoaleatoriedade#Criptografia) dado o contexto de nossos computadores);
>
> 3 - Totalmente secretos.

Para gerar o produto do `dado` com o `dummy`, usaremos o operador lógico `XOR`(^) do Python:

> 0 ^ 0 = 0  
> 0 ^ 1 = 1  
> 1 ^ 0 = 1  
> 1 ^ 1 = 0

Esse operador é utilizado pois possui uma característica interessante para essa aplicação:

> A ^ B = C  
> B ^ C = A  
> C ^ A = B

Que nos remete ao seguinte:

> dado ^ dummy = produto  
> produto ^ dummy = dado  

Dessa forma, ao gerar um par de chaves (`produto` e `dummy`) com a função `encrypt`, poderemos usá-las para obter o valor original do `dado`.

In [6]:
from secrets import token_bytes
from typing import Tuple
from math import ceil


class UnbreakableCrypt:
    @classmethod
    def __generate_random_key(cls, length: int) -> int:
        t_bytes: bytes = token_bytes(length)
        return int.from_bytes(t_bytes, "big")

    @classmethod
    def encrypt(cls, original: str) -> Tuple[int, int]:
        original_bytes: bytes = original.encode()
        dummy: int = cls.__generate_random_key(len(original_bytes))
        original_key: int = int.from_bytes(original_bytes, "big")
        encrypted: int = original_key ^ dummy
        return dummy, encrypted

    @classmethod
    def decrypt(cls, key_1: int, key_2: int) -> str:
        decrypted: int = key_1 ^ key_2
        temp: bytes = decrypted.to_bytes(ceil((decrypted.bit_length()) / 8), "big")
        return temp.decode()

O parâmetro `"big"` passado para as funções `from_bytes` e `to_bytes` indicam qual o bit mais significativo (`endianness`) dos bytes gerados ou convertidos.

Além disso, a função `ceil` tem como função arredondar "para cima" o valor, de forma a evitar o erro por um (`off-by-one`). Aqui o autor usa uma outra alternativa para obter o mesmo resultado: uma soma com o número 7 e então a divisão inteira (`//`).

Abaixo podemos ver se nossa criptografia funciona:

In [13]:
dado = "O rato roeu a roupa do rei de Roma"
key_1, key_2 = UnbreakableCrypt.encrypt(dado)
assert dado == UnbreakableCrypt.decrypt(key_1, key_2)

dado = "1, 2, 3 indiozinhos, 4, 5, 6 indiozinhos"
key_1, key_2 = UnbreakableCrypt.encrypt(dado)
assert dado == UnbreakableCrypt.decrypt(key_1, key_2)

print("A criptografia funciona!")

A criptografia funciona!
